In [1]:
import pandas as pd


In [23]:
def concat_lists(row):
    return ','.join([row[x] for x in row.index])

df = pd.read_table(
    '/projects/ps-yeolab3/bay001/from_yan/marker_genes_scPNM_project/markers_PNMproject.csv',
    sep=','
).T
df['combined'] = df.apply(concat_lists, axis=1)
for x in range(16):
    del df[x]
df.reset_index(inplace=True)
df

,index,combined
0,iPSC,"POU5F1,LIN28A,SOX2,KLF4,DPPA4,CENPK,TTK,SLC7A3..."
1,NPC,"PAX6,CRIP2,TKTL1,ZEB2,PAX3,CDH10,S100B,EN2,SHC..."
2,MN,"STMN2,ISL1,SLIT2,RTN1,MEIS1,MEIS2,MAPT,LIX1,LP..."


In [24]:
def explode(df, delim, col_to_split, cols_to_keep):
    """
    explodes a dataframe by splitting a column on a delimiter, and 
    producing one row for each split. 
    """
    cols_to_keep_list = [y.name for y in cols_to_keep]
    dx = pd.DataFrame(df[col_to_split].str.split(delim).tolist(),index=cols_to_keep).stack()
    dx = dx.reset_index()[[0] + cols_to_keep_list]
    dx.columns = [col_to_split] + cols_to_keep_list
    return dx

In [29]:
exploded = explode(df=df, delim=',', col_to_split='combined', cols_to_keep=[df['index']])
exploded = exploded[['index','combined']]
exploded.columns = ['cell_type','gene_name']
exploded.to_csv(
    '/projects/ps-yeolab4/NCRCRG/refs/cell_markers/markers_PNMproject.tsv',
    sep='\t',
    index=False,
    header=True
)

In [26]:
df['index']

0    iPSC
1     NPC
2      MN
Name: index, dtype: object

In [42]:
idr = pd.read_table(
    '/home/bay001/projects/michelle_msi_20180420/permanent_data/03_IDR/HDMB03_Msi1/results/01v02.idr.out',
    names=['chrom','idrstart','idrend','name','score','strand','score1','score2','peak1start','peak1end','peak1score','peak2start','peak2end','peak2score']
)
idr.head()

,chrom,idrstart,idrend,name,score,strand,score1,score2,peak1start,peak1end,peak1score,peak2start,peak2end,peak2score
0,chr14,58331926,58332013,.,1000,-,5.00,5.00,58331933,58332003,0.00288,58331926,58332013,0.00445
1,chr14,57284024,57284245,.,1000,+,5.00,5.00,57284043,57284233,0.00433,57284024,57284245,0.00239
2,chr14,58332640,58332873,.,1000,-,5.00,5.00,58332645,58332836,0.00372,58332640,58332873,0.00331
3,chr14,58332318,58332412,.,1000,-,5.00,5.00,58332332,58332412,0.00156,58332318,58332409,0.00291
4,chr4,141072243,141072347,.,1000,-,4.25,4.77,141072253,141072347,0.00279,141072243,141072327,0.00066


In [43]:
idrbed = pd.read_table(
    '/home/bay001/projects/michelle_msi_20180420/permanent_data/03_IDR/HDMB03_Msi1/results/01v02.idr.out.bed',
    names=['chrom','start','end','name','score','strand']
)
print(idrbed.shape)
idrbed.head()

(269, 6)


,chrom,start,end,name,score,strand
0,chr16,85645197,85645301,.,.,+
1,chr16,85645225,85645313,.,.,+
2,chr1,864176,864220,.,.,+
3,chr1,864123,864148,.,.,+
4,chr1,864148,864191,.,.,+


In [47]:
merged = pd.merge(
    idr, idrbed,
    how='inner',
    left_on=['chrom','peak1start'],
    right_on=['chrom','start']
)
print(merged.shape)
merged.head()

(93, 19)


,chrom,idrstart,idrend,name_x,score_x,strand_x,score1,score2,peak1start,peak1end,peak1score,peak2start,peak2end,peak2score,start,end,name_y,score_y,strand_y
0,chr14,58331926,58332013,.,1000,-,5.00,5.00,58331933,58332003,0.00288,58331926,58332013,0.00445,58331933,58332003,.,.,-
1,chr14,57284024,57284245,.,1000,+,5.00,5.00,57284043,57284233,0.00433,57284024,57284245,0.00239,57284043,57284125,.,.,+
2,chr14,58332640,58332873,.,1000,-,5.00,5.00,58332645,58332836,0.00372,58332640,58332873,0.00331,58332645,58332665,.,.,-
3,chr4,141072243,141072347,.,1000,-,4.25,4.77,141072253,141072347,0.00279,141072243,141072327,0.00066,141072253,141072267,.,.,-
4,chr12,62996903,62997073,.,1000,+,5.00,5.00,62996910,62997027,0.00267,62996903,62997073,0.00225,62996910,62997027,.,.,+


In [46]:
merged = pd.merge(
    idr, idrbed,
    how='inner',
    left_on=['chrom','peak2start'],
    right_on=['chrom','start']
)
print(merged.shape)
merged.head()

(93, 19)


,chrom,idrstart,idrend,name_x,score_x,strand_x,score1,score2,peak1start,peak1end,peak1score,peak2start,peak2end,peak2score,start,end,name_y,score_y,strand_y
0,chr14,58331926,58332013,.,1000,-,5.00,5.00,58331933,58332003,0.00288,58331926,58332013,0.00445,58331926,58331934,.,.,-
1,chr14,57284024,57284245,.,1000,+,5.00,5.00,57284043,57284233,0.00433,57284024,57284245,0.00239,57284024,57284103,.,.,+
2,chr14,58332640,58332873,.,1000,-,5.00,5.00,58332645,58332836,0.00372,58332640,58332873,0.00331,58332640,58332675,.,.,-
3,chr14,58332318,58332412,.,1000,-,5.00,5.00,58332332,58332412,0.00156,58332318,58332409,0.00291,58332318,58332398,.,.,-
4,chr4,141072243,141072347,.,1000,-,4.25,4.77,141072253,141072347,0.00279,141072243,141072327,0.00066,141072243,141072270,.,.,-
